# **Cutting stock problem**

The cutting stock problem is a combinatorial optimization problem that arises in many industrial applications. The problem is to determine the minimum number of cuts needed to cut a given set of pieces of material, such as pipes, rods, rolls, etc., of standard length, such as a roll, a rod, etc., in order to minimize the waste material generated. The problem is NP-hard, and it is a special case of the more general bin packing problem.

Math model:



### Indices
- $n$: Index for small rolls
- $m$: Index for large rolls

### Parameters
- $N$: Total number of different roll widths demand
- $W_n$: Width of small roll $n$
- $roll\_width$: Width of large roll (same for all large rolls)
- $D_n$: Demand for small roll $n$

### Decision Variables
- $X_{nm}$: Binary variable, 1 if small roll $n$ comes from large roll $m$, 0 otherwise
- $Y_m$: Binary variable, 1 if large roll $m$ is needed, 0 otherwise

### Formulation

#### Objective Function
Minimize the number of large rolls used:

$ \min \sum_{m} Y_m $

#### Constraints

1. **Demand satisfaction**: Each small roll \( n \) must be cut from exactly one large roll:

$ \sum_{m} X_{nm} = D_n \quad \forall n $

2. **Width constraints**: The total width of small rolls cut from a large roll \( m \) must not exceed the width of the large roll:

$ \sum_{n} W_n X_{nm} \leq \text{\texttt{roll\_width}} \cdot Y_m \quad \forall m $

3. **Binary constraints**: The decision variables \( X_{nm} \) and \( Y_m \) are binary:
\[ X_{nm} \in \{0, 1\} \quad \forall n, m \]
\[ Y_m \in \{0, 1\} \quad \forall m \]



# code

In [13]:
import pulp
class CuttingStock:
    def __init__(self, N, M, w, roll_width, d):
        self.N = N
        self.w = w
        self.M = M
        self.roll_width = roll_width
        self.d = d

        #initialize the model
        self.problem = pulp.LpProblem("cutting stock", pulp.LpMinimize)

        #DVs
        self.x = pulp.LpVariable.dicts("x", [(n, m) for n in self.N for m in self.M], cat=pulp.LpBinary)
        self.y = pulp.LpVariable.dicts("y", self.M, cat=pulp.LpBinary)
    
    def model(self):
        #objective function
        self.problem += pulp.lpSum([self.y[m] for m in self.M])

        #constraints
        for n in self.N:
            self.problem += pulp.lpSum([self.x[n, m] for m in self.M]) == self.d[n]

        for m in self.M:
            self.problem += pulp.lpSum([self.w[n] * self.x[n, m] for n in self.N]) <= self.roll_width * self.y[m]

    def solve(self):
        self.problem.solve()
        # Use CPLEX solver
        # self.problem.solve(pulp.CPLEX_PY(msg=True))
        print("Status:", pulp.LpStatus[self.problem.status])
        
        # Print the results
        print("Objective Value:", pulp.value(self.problem.objective))
        for v in self.problem.variables():
            if v.varValue > 0:
                print(v.name, "=", v.varValue)

    

In [10]:
# Example 1
# parameters
N = range(5)  # 5 small rolls
M = range(3)  # 3 large rolls
w = [10, 20, 30, 40, 50]  # Widths of small rolls
roll_width = 100  # Width of large roll
d = [1, 1, 1, 1, 1]  # Demand for each small roll

# Initialize the CuttingStock problem
cutting_stock_problem = CuttingStock(N, M, w, roll_width, d)

# Define the model
cutting_stock_problem.model()

# Solve the model
cutting_stock_problem.solve()

C:\Users\mrkha\AppData\Roaming\Python\Python311\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Version identifier: 22.1.1.0 | 2023-06-15 | d64d5bd77
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve modified 3 coefficients.
Reduced MIP has 8 rows, 18 columns, and 33 nonzeros.
Reduced MIP has 18 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Found incumbent of value 3.000000 after 0.00 sec. (0.07 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 8 rows, 18 columns, and 33 nonzeros.
Reduced MIP has 18 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 20.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integ

In [15]:
#exambple 2
import random
N = range(500)  # 500 small rolls
M = range(200)  # 200 large rolls
w = [random.randint(5, 30) for _ in N]  # Random widths between 5 and 30 for small rolls
roll_width = 100  # Width of large roll
d = [random.randint(1, 3) for _ in N]  # Random demand between 1 and 3 for each small roll

# Initialize the CuttingStock problem
cutting_stock_problem = CuttingStock(N, M, w, roll_width, d)

# Define the model
cutting_stock_problem.model()

# Solve the model
cutting_stock_problem.solve()

NO RESULT AFTER 10 HOUR OF RUNNING

# COLUMN GENERATION

Column generation is an advanced method for solving large-scale linear programming problems, particularly useful for problems like the cutting stock problem. Let's start by formulating the problem using column generation:


## Indices
$ p $ for patterns

## Parameters
$ P $ = number of patterns currently under consideration

$ A_{ip} $ = number of item type \( i \) cut in pattern \( p \)

$ d_i $ = demand for item type \( i \)

$ w_i $ = width of item type \( i \)

$ W $ = width of large roll

## Decision Variables
$ X_p $ = number of large rolls cut according to pattern \( p \)

## Objective Function
Minimize the total number of large rolls used:

$ \min \sum_{p \in P} X_p $

## Constraints

### Demand Satisfaction:
Each item's demand must be met:

$ \sum_{p \in P} A_{ip} X_p \geq d_i \quad \forall i $

### Non-negativity:

$ X_p \geq 0 \quad \forall p $


# Steps for Column Generation
- **Initialization**: Start with a set of initial patterns (e.g., one pattern per item type).
- **Restricted Master Problem (RMP)**: Solve the linear program with the current set of patterns.
- **Subproblem**: Generate new patterns by solving a knapsack problem to determine if there are patterns that can improve the objective function.
- **Iteration**: Add new patterns to the RMP and resolve. Repeat until no improving patterns can be found.

# code

In [17]:
import pulp
import random

class CuttingStock:
    def __init__(self, N, w, roll_width, d):
        self.N = N  # Number of different item types
        self.w = w  # Width of each item type
        self.roll_width = roll_width  # Width of the large roll
        self.d = d  # Demand for each item type
        self.patterns = self.initial_patterns()  # Initial set of patterns

        # Initialize the RMP
        self.rmp = pulp.LpProblem("cutting_stock_RMP", pulp.LpMinimize)
        self.X = pulp.LpVariable.dicts("X", range(len(self.patterns)), lowBound=0, cat=pulp.LpInteger)
    
    def initial_patterns(self):
        """
        Let's take an example with 5 different item types with widths [5, 10, 20, 25, 30] and a large roll width of 100.
        patterns:
        [[20, 0, 0, 0, 0],
        [0, 10, 0, 0, 0],
        [0, 0, 5, 0, 0],
        [0, 0, 0, 4, 0],
        [0, 0, 0, 0, 3]]
        """
        # Generate an initial set of patterns (one pattern per item type)
        patterns = []
        for i in range(len(self.w)):
            pattern = [0] * len(self.w)
            pattern[i] = self.roll_width // self.w[i]
            patterns.append(pattern)
        return patterns
    
    def model_rmp(self):
        # Objective function
        self.rmp += pulp.lpSum([self.X[p] for p in range(len(self.patterns))])

        # Demand satisfaction constraints
        self.constraints = []
        for i in range(len(self.w)):
            constraint = pulp.lpSum([self.patterns[p][i] * self.X[p] for p in range(len(self.patterns))]) >= self.d[i]
            self.constraints.append(constraint)
            self.rmp += constraint
    
    def solve_rmp(self):
        self.rmp.solve()
        print("Status:", pulp.LpStatus[self.rmp.status])
        print("Objective Value:", pulp.value(self.rmp.objective))
        for v in self.rmp.variables():
            if v.varValue > 0:
                print(v.name, "=", v.varValue)

    def get_dual_values(self):
        dual_values = [constraint.pi for constraint in self.constraints]
        print("Dual Values:", dual_values)  # Added print statement for debugging
        return dual_values
    
    def solve_knapsack(self, dual_values):
        # Solve knapsack problem to find new pattern
        knapsack = pulp.LpProblem("Knapsack", pulp.LpMaximize)
        y = pulp.LpVariable.dicts("y", range(len(self.w)), lowBound=0, cat=pulp.LpInteger)

        # Objective function (dual values are used here)
        knapsack += pulp.lpSum([dual_values[i] * y[i] for i in range(len(self.w))])

        # Constraint (total width should not exceed roll width)
        knapsack += pulp.lpSum([self.w[i] * y[i] for i in range(len(self.w))]) <= self.roll_width

        knapsack.solve()
        print("Knapsack Status:", pulp.LpStatus[knapsack.status])  # Added print statement for debugging
        
        new_pattern = [int(pulp.value(y[i])) if pulp.value(y[i]) is not None else 0 for i in range(len(self.w))]
        print("New Pattern:", new_pattern)  # Added print statement for debugging
        
        # Check if a valid new pattern is found
        if sum(new_pattern) == 0:
            print("No new pattern found.")
        else:
            print("New Pattern:", new_pattern)
        
        return new_pattern
    
    def add_new_pattern(self, new_pattern):
        # Add the new pattern to the list of patterns
        self.patterns.append(new_pattern)
        new_index = len(self.patterns) - 1
        self.X[new_index] = pulp.LpVariable(f"X_{new_index}", lowBound=0, cat=pulp.LpInteger)

        # Add the new pattern's constraints to the RMP
        for i in range(len(self.w)):
            self.constraints[i].addterm(new_pattern[i], self.X[new_index])
    
    def column_generation(self):
        self.model_rmp()
        iteration = 0  # Added iteration counter
        while True:
            print(f"Iteration {iteration}")  # Added print statement for debugging
            self.solve_rmp()
            dual_values = self.get_dual_values()

            new_pattern = self.solve_knapsack(dual_values)
            if sum(new_pattern) == 0:
                break  # No new pattern can improve the objective

            self.add_new_pattern(new_pattern)
            iteration += 1  # Increment iteration counter

        print("Final Objective Value:", pulp.value(self.rmp.objective))
    
    def print_patterns(self):
        for idx, pattern in enumerate(self.patterns):
            print(f"Pattern {idx}: {pattern}")
    
    def print_solution(self):
        print("Final Patterns and Usage:")
        for p in range(len(self.patterns)):
            if pulp.value(self.X[p]) > 0:
                print(f"Pattern {p}: {self.patterns[p]} used {pulp.value(self.X[p])} times")


In [14]:
# Parameters
N = range(5)  # Number of different item types
w = [5, 10, 20, 25, 30]  # Width of each item type
roll_width = 100  # Width of large roll
d = [5, 7, 3, 9, 6]  # Demand for each item type

# Initialize the CuttingStock problem
cutting_stock_problem = CuttingStock(N, w, roll_width, d)

# Print initial patterns
cutting_stock_problem.print_patterns()

# Setup and solve the RMP
cutting_stock_problem.model_rmp()
cutting_stock_problem.solve_rmp()

# Get dual values from RMP
dual_values = cutting_stock_problem.get_dual_values()
print("Dual Values:", dual_values)

# Solve knapsack problem to find new pattern
new_pattern = cutting_stock_problem.solve_knapsack(dual_values)
print("New Pattern:", new_pattern)

Pattern 0: [20, 0, 0, 0, 0]
Pattern 1: [0, 10, 0, 0, 0]
Pattern 2: [0, 0, 5, 0, 0]
Pattern 3: [0, 0, 0, 4, 0]
Pattern 4: [0, 0, 0, 0, 3]
Status: Optimal
Objective Value: 8.0
X_0 = 1.0
X_1 = 1.0
X_2 = 1.0
X_3 = 3.0
X_4 = 2.0
Dual Values: [0.0, 0.0, 0.0, 0.0, 0.0]
Dual Values: [0.0, 0.0, 0.0, 0.0, 0.0]
Knapsack Status: Optimal
New Pattern: [0, 0, 0, 0, 0]
No new pattern found.
New Pattern: [0, 0, 0, 0, 0]


In [18]:
# Parameters for the larger example
N = range(20)  # 20 different item types
w = [random.randint(5, 30) for _ in N]  # Random widths between 5 and 30 for item types
roll_width = 100  # Width of large roll
d = [random.randint(1, 10) for _ in N]  # Random demand between 1 and 10 for each item type

# Initialize the CuttingStock problem
cutting_stock_problem = CuttingStock(N, w, roll_width, d)

# Print initial patterns
cutting_stock_problem.print_patterns()

# Run column generation
cutting_stock_problem.column_generation()

# Print final solution
cutting_stock_problem.print_solution()

Pattern 0: [7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Pattern 1: [0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Pattern 2: [0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Pattern 3: [0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Pattern 4: [0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Pattern 5: [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Pattern 6: [0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Pattern 7: [0, 0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Pattern 8: [0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Pattern 9: [0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Pattern 10: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Pattern 11: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0]
Pattern 12: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0]
Pattern 13: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0,

In [15]:
# Parameters for the large example
N = range(500)  # 500 small rolls
w = [random.randint(5, 10) for _ in N]  # Random widths between 5 and 30 for small rolls
roll_width = 100  # Width of large roll
d = [random.randint(1, 3) for _ in N]  # Random demand between 1 and 3 for each small roll

# Initialize the CuttingStock problem
cutting_stock_problem = CuttingStock(N, w, roll_width, d)

# Print initial patterns (optional, might be large)
# cutting_stock_problem.print_patterns()

# Run column generation
cutting_stock_problem.column_generation()

Iteration 0
Status: Optimal
Objective Value: 500.0
X_0 = 1.0
X_1 = 1.0
X_10 = 1.0
X_100 = 1.0
X_101 = 1.0
X_102 = 1.0
X_103 = 1.0
X_104 = 1.0
X_105 = 1.0
X_106 = 1.0
X_107 = 1.0
X_108 = 1.0
X_109 = 1.0
X_11 = 1.0
X_110 = 1.0
X_111 = 1.0
X_112 = 1.0
X_113 = 1.0
X_114 = 1.0
X_115 = 1.0
X_116 = 1.0
X_117 = 1.0
X_118 = 1.0
X_119 = 1.0
X_12 = 1.0
X_120 = 1.0
X_121 = 1.0
X_122 = 1.0
X_123 = 1.0
X_124 = 1.0
X_125 = 1.0
X_126 = 1.0
X_127 = 1.0
X_128 = 1.0
X_129 = 1.0
X_13 = 1.0
X_130 = 1.0
X_131 = 1.0
X_132 = 1.0
X_133 = 1.0
X_134 = 1.0
X_135 = 1.0
X_136 = 1.0
X_137 = 1.0
X_138 = 1.0
X_139 = 1.0
X_14 = 1.0
X_140 = 1.0
X_141 = 1.0
X_142 = 1.0
X_143 = 1.0
X_144 = 1.0
X_145 = 1.0
X_146 = 1.0
X_147 = 1.0
X_148 = 1.0
X_149 = 1.0
X_15 = 1.0
X_150 = 1.0
X_151 = 1.0
X_152 = 1.0
X_153 = 1.0
X_154 = 1.0
X_155 = 1.0
X_156 = 1.0
X_157 = 1.0
X_158 = 1.0
X_159 = 1.0
X_16 = 1.0
X_160 = 1.0
X_161 = 1.0
X_162 = 1.0
X_163 = 1.0
X_164 = 1.0
X_165 = 1.0
X_166 = 1.0
X_167 = 1.0
X_168 = 1.0
X_169 = 1.0
X_17 = 1.0
X